In [ ]:
import sqlite3
import numpy as np 
import pandas as pd 
import matplotlib.pyplot as plt 
import sklearn as sk 
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import RandomForestClassifier
from sqlite3 import Error
from sklearn.utils import shuffle
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import balanced_accuracy_score
from sklearn.metrics import confusion_matrix
from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))
database = "../input/FPA_FOD_20170508.sqlite"
test = pd.read_parquet("../input/wildfires-testset/test-wildfires.parquet")
train = pd.read_parquet("../input/wildfirestrain/train-wildfires.parquet")
print (train[['STAT_CAUSE_CODE', 'STAT_CAUSE_DESCR']].drop_duplicates())

In [ ]:
##state included, No cont date, unnormalized, logistic and random forest

def create_connection(db_file):
    """ create a database connection to the SQLite database
        specified by the db_file
    :param db_file: database file
    :return: Connection object or None
    """
    try:
        conn = sqlite3.connect(db_file)
        return conn
    except Error as e:
        print(e)
 
    return None
 



# create a database connection
#conn = create_connection(database)
"""
cur = conn.cursor()
cur.execute('SELECT TABLE_NAME FROM INFORMATION_SCHEMA.TABLES')
print(cur.fetchall())
"""
#print(train.columns)
#df = pd.read_sql_query("SELECT STAT_CAUSE_CODE,FIRE_YEAR,LATITUDE,LONGITUDE,DISCOVERY_DATE,FIRE_SIZE FROM 'Fires' WHERE STATE='CA' ", conn)
#df = df[df.CONT_DATE.notna()]
#df3 = pd.read_sql_query("SELECT STAT_CAUSE_CODE, FIRE_YEAR, LATITUDE, LONGITUDE, DISCOVERY_DATE,FIRE_SIZE, STATE FROM Fires", conn)
df3 = train[['STAT_CAUSE_CODE','FIRE_YEAR', 'LATITUDE', 'LONGITUDE','DISCOVERY_DATE','FIRE_SIZE','STATE']]
test3 = test[['STAT_CAUSE_CODE','FIRE_YEAR', 'LATITUDE', 'LONGITUDE','DISCOVERY_DATE','FIRE_SIZE','STATE']]
#print(len(df3))
df3 = pd.concat([df3,pd.get_dummies(df3['STATE'], prefix='STATE')],axis=1).drop(['STATE'],axis=1)
test3 = pd.concat([test3,pd.get_dummies(test3['STATE'], prefix='STATE')],axis=1).drop(['STATE'],axis=1)
#remove causes 9 and 13 from dataset
df3 = df3[(df3['STAT_CAUSE_CODE'] != 9) & (df3['STAT_CAUSE_CODE'] != 13)]
test3 = test3[(test3['STAT_CAUSE_CODE'] != 9) & (test3['STAT_CAUSE_CODE'] != 13)]
#shuffle dataset
df3 = shuffle(df3)
y = df3.iloc[:,0]
print(type(y))
x = df3.iloc[:,1:]
y_test = test3.iloc[:,0]
X_test = test3.iloc[:,1:]
#print(df.head())
#print(df.STAT_CAUSE_CODE.unique())
#df2 = pd.read_sql_query("SELECT DISTINCT STATE FROM 'Fires'", conn)
#print(df2)
size = len(y)
split = int(size*0.77)
y_train = y.iloc[:split]
X_train = x.iloc[:split]
#y_test = y.iloc[split:]
#X_test = x.iloc[split:]

LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_train, y_train)  
y_pred=LR.predict(X_test)  
print(round(LR.score(X_test,y_test), 4))#0.3131
print(balanced_accuracy_score(y_test,y_pred))#0.0922
print(confusion_matrix(y_test,y_pred))
print(LR.classes_)
"""
[[  407     0     0     0 27273     0     0     0     0     0     0]
 [   35     0     0     0 14647     0     0     0     0     0     0]
 [    6     0     0     0  5213     0     0     0     0     0     0]
 [   15     0     0     0  7661     0     0     0     0     0     0]
 [   21     0     0     0 43008     0     0     0     0     0     0]
 [    5     0     0     0  3329     0     0     0     0     0     0]
 [   39     0     0     0 28053     0     0     0     0     0     0]
 [    3     0     0     0  6077     0     0     0     0     0     0]
 [    2     0     0     0  1055     0     0     0     0     0     0]
 [   13     0     0     0  1427     0     0     0     0     0     0]
 [    2     0     0     0   375     0     0     0     0     0     0]]
 """

RF = RandomForestClassifier(n_estimators=1000, max_depth=10, random_state=0).fit(X_train, y_train)  
y_pred=RF.predict(X_test)  
print(round(RF.score(X_test, y_test), 4)) #0.5127
print(balanced_accuracy_score(y_test, y_pred))#0.2299
print(confusion_matrix(y_test, y_pred))
#0.3845 with no shuffle, and all categories, n_estimators = 1000, max_depth = 10, random_state = 0

"""
mlp = MLPClassifier()
mlp.fit(X_train, y_train)
y_pred=mlp.predict(X_test)
print(round(mlp.score(X_test,y_test),4))#0.3103
print(balanced_accuracy_score(y_test,y_pred))#0.090909
#0.3087
"""

In [ ]:
print(confusion_matrix(y_test,y_pred))
"""
[[22436   737     0     0  3516   382   609     0     0     0     0]
 [ 2609  3607     0     6  7179   624   657     0     0     0     0]
 [ 1116   420     0     0  2914    52   717     0     0     0     0]
 [ 3984   434     0    47  2834    37   340     0     0     0     0]
 [ 3707   983     0     0 35181   189  2969     0     0     0     0]
 [  646    50     0     0  1373  1130   135     0     0     0     0]
 [ 1994  1299     0     3 16008    92  8696     0     0     0     0]
 [ 1036   560     0     0  3665   183   636     0     0     0     0]
 [  487     9     0     0   507     0    54     0     0     0     0]
 [  314   119     0     0   907     0   100     0     0     0     0]
 [   94     5     0     0   263     0    15     0     0     0     0]]
"""

In [ ]:
#state, Contdate, na dropped, unnormalized, logistic regression

df4 = train[['STAT_CAUSE_CODE','FIRE_YEAR', 'LATITUDE', 'LONGITUDE','DISCOVERY_DATE','CONT_DATE','FIRE_SIZE','STATE']]
test4 = test[['STAT_CAUSE_CODE','FIRE_YEAR', 'LATITUDE', 'LONGITUDE','DISCOVERY_DATE','CONT_DATE','FIRE_SIZE','STATE']]
df4 = pd.concat([df4,pd.get_dummies(df4['STATE'], prefix='STATE')],axis=1).drop(['STATE'],axis=1)
test4 = pd.concat([test4,pd.get_dummies(test4['STATE'], prefix='STATE')],axis=1).drop(['STATE'],axis=1)
df4 = df4[(df4['STAT_CAUSE_CODE'] != 9) & (df4['STAT_CAUSE_CODE'] != 13)]
test4 = test4[(test4['STAT_CAUSE_CODE'] != 9) & (test4['STAT_CAUSE_CODE'] != 13)]
df4 = df4.dropna()
test4 = test4.dropna()

df4 = shuffle(df4)
y = df4.iloc[:,0]
x = df4.iloc[:,1:]
y_test = test4.iloc[:,0]
X_test = test4.iloc[:,1:]

size = len(y)
split = int(size*0.77)
y_train = y.iloc[:split]
X_train = x.iloc[:split]

LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_train, y_train)  
y_pred=LR.predict(X_test)  
print(round(LR.score(X_test,y_test), 4))#0.3014
print(balanced_accuracy_score(y_test,y_pred))#0.0909
print(confusion_matrix(y_test,y_pred))
"""
[[22581     0     0     0     0     0     0     0     0     0     0]
 [ 5341     0     0     0     0     0     0     0     0     0     0]
 [ 2493     0     0     0     0     0     0     0     0     0     0]
 [ 5608     0     0     0     0     0     0     0     0     0     0]
 [18534     0     0     0     0     0     0     0     0     0     0]
 [  849     0     0     0     0     0     0     0     0     0     0]
 [14518     0     0     0     0     0     0     0     0     0     0]
 [ 2888     0     0     0     0     0     0     0     0     0     0]
 [  983     0     0     0     0     0     0     0     0     0     0]
 [  833     0     0     0     0     0     0     0     0     0     0]
 [  301     0     0     0     0     0     0     0     0     0     0]]
"""
print(LR.classes_)

In [ ]:
#state, contdate, na dropped, normalized, logistic regression

mean = df4.mean()
mean.iloc[0]=0
drange = df4.max()-df4.min()
drange[0]=1

dnorm = (df4-mean)/drange
tnorm = (test4-mean)/drange
dnorm.iloc[7:]=df4.iloc[7:]
tnorm.iloc[7:]=test4.iloc[7:]

y = dnorm.iloc[:,0]
x = dnorm.iloc[:,1:]
y_test = tnorm.iloc[:,0]
X_test = tnorm.iloc[:,1:]

size = len(y)
split = int(size*0.77)
y_train = y.iloc[:split]
X_train = x.iloc[:split]

LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial').fit(X_train, y_train)  
y_pred=LR.predict(X_test)  
print(round(LR.score(X_test,y_test), 4))#0.3014
print(balanced_accuracy_score(y_test,y_pred))#0.0909
print(confusion_matrix(y_test,y_pred))
"""
[[22581     0     0     0     0     0     0     0     0     0     0]
 [ 5341     0     0     0     0     0     0     0     0     0     0]
 [ 2493     0     0     0     0     0     0     0     0     0     0]
 [ 5608     0     0     0     0     0     0     0     0     0     0]
 [18530     0     0     0     2     0     0     0     0     0     2]
 [  849     0     0     0     0     0     0     0     0     0     0]
 [14517     0     0     0     1     0     0     0     0     0     0]
 [ 2888     0     0     0     0     0     0     0     0     0     0]
 [  983     0     0     0     0     0     0     0     0     0     0]
 [  833     0     0     0     0     0     0     0     0     0     0]
 [  301     0     0     0     0     0     0     0     0     0     0]]
 """

In [ ]:
#state, contdate included, extra column, unnormalized, logistic regression
df5 = train[['STAT_CAUSE_CODE','FIRE_YEAR', 'LATITUDE', 'LONGITUDE','DISCOVERY_DATE','CONT_DATE','FIRE_SIZE','STATE']]
test5 = test[['STAT_CAUSE_CODE','FIRE_YEAR', 'LATITUDE', 'LONGITUDE','DISCOVERY_DATE','CONT_DATE','FIRE_SIZE','STATE']]
df5 = pd.concat([df5,pd.get_dummies(df5['STATE'], prefix='STATE')],axis=1).drop(['STATE'],axis=1)
test5 = pd.concat([test5,pd.get_dummies(test5['STATE'], prefix='STATE')],axis=1).drop(['STATE'],axis=1)
df5 = df5[(df5['STAT_CAUSE_CODE'] != 9) & (df5['STAT_CAUSE_CODE'] != 13)]
test5 = test5[(test5['STAT_CAUSE_CODE'] != 9) & (test5['STAT_CAUSE_CODE'] != 13)]
df5['burn_time'] = df5['CONT_DATE'] - df5['DISCOVERY_DATE']
df5['burn_time_notna'] = df5.burn_time.notna().astype(int)
df5 = df5.drop(['CONT_DATE','DISCOVERY_DATE'], axis = 1)
test5['burn_time'] = test5['CONT_DATE']-test5['DISCOVERY_DATE']
test5['burn_time_notna'] = test5.burn_time.notna().astype(int)
test5 = test5.drop(['CONT_DATE', 'DISCOVERY_DATE'], axis = 1)
df5 = df5.fillna(-1)
test5 = test5.fillna(-1)
print(df5.columns)

In [ ]:
#print(df5[df5.isna().any(axis=1)])
y = df5.iloc[:,0]
x = df5.iloc[:,1:]
y_test = test5.iloc[:,0]
X_test = test5.iloc[:,1:]

size = len(y)
split = int(size*0.77)
y_train = y.iloc[:split]
X_train = x.iloc[:split]

mlp = MLPClassifier()
mlp.fit(X_train, y_train)
y_pred=mlp.predict(X_test)
print(round(mlp.score(X_test,y_test),4))#0.4981
print(balanced_accuracy_score(y_test,y_pred))#0.2242
print(confusion_matrix(y_test,y_pred))
#without discovery date
"""
[[21345   794     0     5  3177   941  1318   100     0     0     0]
 [ 2368  3632     0     6  6840   748   962   126     0     0     0]
 [  996   465     0     4  2734    75   915    30     0     0     0]
 [ 3810   471     0     4  2486    78   771    56     0     0     0]
 [ 3231  1194     1     9 33174   539  4690   191     0     0     0]
 [  806    54     0     0  1283   974   214     3     0     0     0]
 [ 2364  1404     0     4 14279   281  9705    54     0     1     0]
 [  914   613     0     0  3330   288   697   238     0     0     0]
 [  418     7     0     0   331     1   155   145     0     0     0]
 [  286    84     0     0   877     0   185     8     0     0     0]
 [   74     6     0     0   196     0    44    57     0     0     0]]
"""
LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial',max_iter=1000).fit(X_train, y_train)  
y_pred=LR.predict(X_test)
#first set without discovery date, second set with
print(round(LR.score(X_test,y_test), 4))#0.4311#3131
print(balanced_accuracy_score(y_test,y_pred))#0.1513#0.0922
print(confusion_matrix(y_test,y_pred))
"""
[[21160     2     0     0  6370     0   148     0     0     0     0]
 [ 5566     1     0     0  9066     0    49     0     0     0     0]
 [ 1499     1     0     0  3693     0    26     0     0     0     0]
 [ 4369     0     0     0  3276     0    31     0     0     0     0]
 [ 4517     0     0     0 38444     0    68     0     0     0     0]
 [  364     0     0     0  2961     0     9     0     0     0     0]
 [ 3488     0     0     0 24430     0   174     0     0     0     0]
 [ 1579     0     0     0  4473     0    28     0     0     0     0]
 [  492     0     0     0   564     0     1     0     0     0     0]
 [  377     0     0     0  1056     0     7     0     0     0     0]
 [  121     0     0     0   256     0     0     0     0     0     0]]
"""
"""
[[  415     0     0     0 27265     0     0     0     0     0     0]
 [   36     0     0     0 14646     0     0     0     0     0     0]
 [    7     0     0     0  5212     0     0     0     0     0     0]
 [   15     0     0     0  7661     0     0     0     0     0     0]
 [   23     0     0     0 43006     0     0     0     0     0     0]
 [    6     0     0     0  3328     0     0     0     0     0     0]
 [   41     0     0     0 28051     0     0     0     0     0     0]
 [    3     0     0     0  6077     0     0     0     0     0     0]
 [    2     0     0     0  1055     0     0     0     0     0     0]
 [   14     0     0     0  1426     0     0     0     0     0     0]
 [    2     0     0     0   375     0     0     0     0     0     0]]
"""

In [ ]:
#state, contdate, extra column, normalized, logistic regression

mean = df5.mean()
mean.iloc[0]=0
drange = df5.max()-df5.min()
drange[0]=1

dnorm = (df5-mean)/drange
tnorm = (test5-mean)/drange
dnorm.iloc[5:-2]=df5.iloc[5:-2]
tnorm.iloc[5:-2]=test5.iloc[5:-2]

y = dnorm.iloc[:,0]
x = dnorm.iloc[:,1:]
y_test = tnorm.iloc[:,0]
X_test = tnorm.iloc[:,1:]

size = len(y)
split = int(size*0.77)
y_train = y.iloc[:split]
X_train = x.iloc[:split]

mlp = MLPClassifier()
mlp.fit(X_train, y_train)
y_pred=mlp.predict(X_test)
print(round(mlp.score(X_test,y_test),4))#4986
print(balanced_accuracy_score(y_test,y_pred))#0.2188
print(confusion_matrix(y_test,y_pred))
"""
[[21577   734     0     3  3042   782  1367   175     0     0     0]
 [ 2376  3761     0     8  6674   512  1196   155     0     0     0]
 [  983   515     0     4  2708    30   945    34     0     0     0]
 [ 3784   497     0    14  2452    52   819    58     0     0     0]
 [ 3183  1322     0     9 32797   341  5101   276     0     0     0]
 [  889   294     0     1  1279   649   221     1     0     0     0]
 [ 2424  1383     0     1 14019   208  9989    68     0     0     0]
 [  946   669     0     3  3149   213   754   346     0     0     0]
 [  455     6     0     1   236     1   177   181     0     0     0]
 [  277    88     0     1   831     0   231    12     0     0     0]
 [   72     5     0     0   172     0    60    68     0     0     0]]
"""

LR = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial',max_iter=1000).fit(X_train, y_train)  
y_pred=LR.predict(X_test) 
#first set with discovery date
print(round(LR.score(X_test,y_test), 4))##0.3131
print(balanced_accuracy_score(y_test,y_pred))#0.0922
print(confusion_matrix(y_test,y_pred))

"""
[[  416     0     0     0 27264     0     0     0     0     0     0]
 [   36     0     0     0 14646     0     0     0     0     0     0]
 [    7     0     0     0  5212     0     0     0     0     0     0]
 [   16     0     0     0  7660     0     0     0     0     0     0]
 [   24     0     0     0 43004     0     0     0     0     0     1]
 [    6     0     0     0  3328     0     0     0     0     0     0]
 [   42     0     0     0 28050     0     0     0     0     0     0]
 [    3     0     0     0  6077     0     0     0     0     0     0]
 [    2     0     0     0  1055     0     0     0     0     0     0]
 [   14     0     0     0  1426     0     0     0     0     0     0]
 [    2     0     0     0   375     0     0     0     0     0     0]]
"""